In [ ]:
import kaggle
kaggle.api.dataset_download_files(dataset="seryouxblaster764/fgvc-aircraft", path="data/aircraft", unzip=True)
kaggle.api.dataset_download_files(dataset="gpiosenka/100-bird-species", path="data/birds", unzip=True)